<a href="https://colab.research.google.com/github/lisun85/AI-MakerSpace/blob/main/Synthetic_Data_Generation_RAGAS_%26_LangSmith_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [30]:
!pip install -qU ragas==0.2.10

In [32]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [31]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

LangChain API Key:··········


We'll also want to set a project name to make things easier for ourselves.

In [34]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [35]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [36]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [37]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31440    0 31440    0     0   305k      0 --:--:-- --:--:-- --:--:--  307k


In [38]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70299    0 70299    0     0   528k      0 --:--:-- --:--:-- --:--:--  528k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [39]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [46]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [41]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [42]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [48]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/14 [00:00<?, ?it/s]

ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' proper

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/24 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/50 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 26, relationships: 242)

We can save and load our knowledge graphs as follows.

In [49]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 26, relationships: 242)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [50]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [51]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

SingleHopSpecificQuerySynthesizer: This synthesizer generates queries that require a single hop in the knowledge graph, focusing on direct relationships between nodes. For example, if there is a direct connection between "AI in 2023" and "Key Developments," it might produce a question like, "What were the key developments in AI in 2023?" This type of query is straightforward and seeks specific information based on immediate connections.

MultiHopAbstractQuerySynthesizer: This synthesizer creates queries that necessitate multiple hops in the knowledge graph, leading to more abstract or generalized questions. It connects various nodes to derive a broader understanding of the data. An example of a question generated by this synthesizer could be, "How has AI evolved over the years based on various developments?" This requires traversing multiple relationships to form a comprehensive query.

MultiHopSpecificQuerySynthesizer: Similar to the MultiHopAbstractQuerySynthesizer, this synthesizer focuses on generating specific queries that still involve multiple hops. It aims to extract detailed information while navigating through several nodes. For instance, it might generate a question like, "What specific advancements in AI were influenced by the developments in 2023 and 2024?" This type of query combines specificity with the complexity of multiple relationships in the knowledge graph.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [52]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is GPT-4 and why is it significant?,[The ethics of this space remain diabolically ...,GPT-4 was released by OpenAI in March and is c...,single_hop_specifc_query_synthesizer
1,What recent legal action has the New York Time...,"[and software engineer, LLMs are infuriating. ...",The New York Times has launched a landmark law...,single_hop_specifc_query_synthesizer
2,In what ways did 2023 signify a breakthrough y...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was a breakthrough year for Large Languag...,single_hop_specifc_query_synthesizer
3,What insights can you share about OpenAI based...,[the document includes some of the clearest ex...,"In 2023, I wrote extensively about OpenAI, wit...",single_hop_specifc_query_synthesizer
4,What role does Princeton play in the current l...,[Prompt driven app generation is a commodity a...,Princeton is one of the 18 organizations that ...,single_hop_specifc_query_synthesizer
5,How have LLM context lengths evolved and what ...,[<1-hop>\n\nPrompt driven app generation is a ...,LLM context lengths have significantly evolved...,multi_hop_abstract_query_synthesizer
6,What advancements in AI technology have been f...,[<1-hop>\n\nPrompt driven app generation is a ...,Apple's MLX library has significantly advanced...,multi_hop_abstract_query_synthesizer
7,What are the training cost and energy efficien...,[<1-hop>\n\nthat. DeepSeek v3 is a huge 685B p...,DeepSeek v3 was trained at an estimated cost o...,multi_hop_abstract_query_synthesizer
8,How do LLMs utilize Python in their developmen...,"[<1-hop>\n\nand software engineer, LLMs are in...",LLMs utilize Python as one of the programming ...,multi_hop_specific_query_synthesizer
9,How has Meta's approach to training data and m...,[<1-hop>\n\nAnother common technique is to use...,Meta's approach to training data has evolved s...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [53]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [54]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Microsoft do with LLMs?,[The ethics of this space remain diabolically ...,Microsoft’s Phi-2 claims to have used '14 days...,single_hop_specifc_query_synthesizer
1,How does Python programming relate to the capa...,"[and software engineer, LLMs are infuriating. ...",Python programming is particularly well-suited...,single_hop_specifc_query_synthesizer
2,What we know about GPT-4 and how it fit in LLMs?,[Simon Willison’s Weblog Subscribe Stuff we fi...,"We don’t yet know how to build GPT-4, but it i...",single_hop_specifc_query_synthesizer
3,What are the implications of DALL-E 3 on the f...,[the document includes some of the clearest ex...,The implications of DALL-E 3 on the future of ...,single_hop_specifc_query_synthesizer
4,What are the code generation capabilities of L...,[<1-hop>\n\nThe ethics of this space remain di...,LLMs have demonstrated remarkable code generat...,multi_hop_abstract_query_synthesizer
5,What are the code generation capabilities of L...,[<1-hop>\n\nThe ethics of this space remain di...,LLMs have demonstrated significant code genera...,multi_hop_abstract_query_synthesizer
6,What are the capabilities of LLMs in code gene...,[<1-hop>\n\nThe ethics of this space remain di...,LLMs have demonstrated significant capabilitie...,multi_hop_abstract_query_synthesizer
7,What are the implications of gullibility in la...,"[<1-hop>\n\nand software engineer, LLMs are in...",The implications of gullibility in language mo...,multi_hop_abstract_query_synthesizer
8,What are the implications of Meta's advancemen...,[<1-hop>\n\nThe ethics of this space remain di...,"Meta's advancements in LLMs, particularly with...",multi_hop_specific_query_synthesizer
9,What advancements have been made by Anthropic ...,[<1-hop>\n\nThe ethics of this space remain di...,"In 2024, Anthropic has made significant advanc...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [55]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years - v2"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years v2"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [56]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [57]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [59]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [60]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [61]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [62]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [63]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [64]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [65]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are a vague term in the context of AI, often referring to AI systems that can act on behalf of users, similar to a travel agent. However, there are different interpretations, with some viewing agents as LLMs that can utilize tools in problem-solving. The term lacks a clear and widely accepted definition, leading to frustration among those discussing it.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [69]:
eval_llm = ChatOpenAI(model="gpt-4o-mini")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [70]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [71]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'glossy-trip-38' at:
https://smith.langchain.com/o/39d4534e-1cea-5467-9fe3-b0439640aa8b/datasets/876fd786-a3a6-435c-ad97-3cfc7e3ed191/compare?selectedSessions=64230215-332f-4422-94aa-abd6127a9c5e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"What advancements in AI technologies, particul...",I don't know.,None,"In 2024, significant advancements in AI techno...",0,0,0,0.575125,c6016549-d647-494c-934b-c6b8ddde28a4,6a49a76d-1487-42e8-bbf1-489bb0465e98
1,What significant advancements in Large Languag...,"In 2024, significant advancements in Large Lan...",None,"In 2024, significant advancements in Large Lan...",0,0,0,1.773446,8277c9da-a8f3-44e7-9e89-cd9408f508a2,d1a37368-8747-4f54-bb83-868e5ad49730
2,What advancements have been made by Anthropic ...,I don't know.,None,"In 2024, Anthropic has made significant advanc...",0,0,1,0.814110,853aa0be-5e37-4597-8862-36a777f6dfd8,674542bb-8615-4d39-a199-e219492fe210
3,What are the implications of Meta's advancemen...,The implications of Meta's advancements in LLM...,None,"Meta's advancements in LLMs, particularly with...",1,1,0,3.923265,272e390e-00ef-49ea-986c-06fbfdbdbcc5,243f237f-e3a6-4e0a-80d1-6d0680e59156
4,What are the implications of gullibility in la...,The implications of gullibility in language mo...,None,The implications of gullibility in language mo...,1,1,0,4.209824,942d4c8d-9671-4108-ae46-729a53f27d4f,c5da062a-6724-45f4-bf17-60a02d249ae0
5,What are the capabilities of LLMs in code gene...,LLMs are notably good at writing code and can ...,None,LLMs have demonstrated significant capabilitie...,1,1,1,2.398673,4541b378-2a89-4766-b0c1-4bd5417d6d58,1ad1af9d-6ab8-4445-b059-0f5b062e5b99
6,What are the code generation capabilities of L...,LLMs are notably skilled at writing code and c...,None,LLMs have demonstrated significant code genera...,1,1,1,1.699054,3f1458ed-31c0-4b18-b5dd-cfe5469e4d10,44d3efca-a5c4-4e6a-ba65-ae28bc1aaf2f
7,What are the code generation capabilities of L...,LLMs are noted for being spookily good at writ...,None,LLMs have demonstrated remarkable code generat...,1,1,1,2.549661,f9aa27f0-be79-4955-b783-06a603b07d52,cbb0074d-3a88-4121-858c-87716a55bfaf
8,What are the implications of DALL-E 3 on the f...,I don't know.,None,The implications of DALL-E 3 on the future of ...,0,1,1,0.528853,fe357357-2491-4c28-b9b2-8ab87dd13f3c,15164b4c-a268-410c-8472-e3e1441b5eb7
9,What we know about GPT-4 and how it fit in LLMs?,GPT-4 was initially considered a significant b...,None,"We don’t yet know how to build GPT-4, but it i...",0,0,0,1.719138,5115e8db-6966-49fd-a9d8-11a73ccfaed8,872fe0e1-1156-4e69-9632-1b9e6838d582


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [72]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [73]:
rag_documents = docs

In [74]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

Modifying the chunk size in your application can significantly impact performance for several reasons:
1. Contextual Relevance: A larger chunk size allows the model to process more information at once, which can enhance the contextual understanding of the data. This is particularly important in tasks like question answering, where the model needs to grasp the broader context to generate accurate responses. If the chunk size is too small, the model may miss critical information that spans across chunks, leading to less coherent or relevant answers.

2. Computational Efficiency: Larger chunks can reduce the number of processing iterations required, as the model handles more data in a single pass. This can lead to faster processing times and lower computational overhead. However, if the chunks are too large, it may lead to memory issues or slower processing due to the increased complexity of handling larger data sets. Finding the right balance is crucial for optimizing performance.

3. Trade-off Between Detail and Generalization: Smaller chunks may provide more granular detail, which can be beneficial for specific queries that require precise information. However, this can also lead to an increase in the number of queries and the overall workload for the model. Conversely, larger chunks may generalize the information, which can be useful for broader queries but may sacrifice some detail. Adjusting the chunk size allows you to tailor the model's performance based on the specific requirements of your application, whether that be detail-oriented tasks or broader contextual understanding.

There will be optimal chunking size dependent on agent's task. Best rule of thumb is what Robert Says best "test it!"


In [88]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

1. Quality of Representations: Different embedding models capture semantic relationships and contextual nuances in varying degrees. A more advanced model, such as text-embedding-3-large, may provide richer and more accurate embeddings compared to a simpler model like text-embedding-3-small. This can lead to better understanding and retrieval of relevant information during query processing, ultimately improving the quality of responses generated by the model.

2. Dimensionality and Complexity: The choice of embedding model also impacts the dimensionality of the embeddings. Higher-dimensional embeddings can capture more complex relationships but may also introduce challenges such as increased computational load and potential overfitting. Conversely, lower-dimensional embeddings may be faster to compute and easier to manage but might lack the depth needed for nuanced understanding. Adjusting the embedding model allows you to balance these factors based on the specific needs of your application.

3. Task-Specific Performance: Different tasks may benefit from different types of embeddings. For instance, if your application requires understanding subtle distinctions in meaning or context, a more sophisticated embedding model may yield better results. On the other hand, for tasks that are less sensitive to such nuances, a simpler model might suffice. By modifying the embedding model, you can tailor the performance of your application to align with the specific requirements of the tasks it needs to perform, enhancing overall effectiveness.



In [89]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [90]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [91]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [92]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents, man, are these super vague AI systems that people talk about as if they can just go off and act on your behalf, like a travel agent or a digital assistant. But here\'s the kicker: the term is tossed around without a clear definition, so when someone says they\'re building "agents," it doesn\'t really tell you much. There\'s a lot of excitement, but actual production examples are few and far between. Plus, there\'s this whole issue of gullibility—these systems can struggle to distinguish truth from fiction, which is a major roadblock. So yeah, agents are cool in theory, but in practice? Still waiting on that magic to happen!'

Finally, we can evaluate the new chain on the same test set!

In [93]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'warm-substance-16' at:
https://smith.langchain.com/o/39d4534e-1cea-5467-9fe3-b0439640aa8b/datasets/876fd786-a3a6-435c-ad97-3cfc7e3ed191/compare?selectedSessions=9cbafedd-0424-49c6-bc5b-9b41ef35ff0e




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 403 - {'error': {'message': 'Project `proj_u9CxAA2h51we3e5QHBbW0gF8` does not have access to model `text-embedding-ada-002`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/langsmith/evaluation/_runner.py", line 1855, in _forward
    fn(
  File "/usr/local/lib/python3.11/dist-packages/langsmith/run_helpers.py", line 621, in wrapper
    raise e
  File "/usr/local/lib/python3.11/dist-packages/langsmith/run_helpers.py", line 618, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"What advancements in AI technologies, particul...","Yo, in 2024, the AI scene got a serious upgrad...",None,"In 2024, significant advancements in AI techno...",1,0,1,2.774281,c6016549-d647-494c-934b-c6b8ddde28a4,0b54e95d-ab6b-4261-877d-e828013f98af
1,What significant advancements in Large Languag...,"Yo, 2024 was a game-changer for Large Language...",None,"In 2024, significant advancements in Large Lan...",1,1,1,5.685203,8277c9da-a8f3-44e7-9e89-cd9408f508a2,f15fb241-5bdf-4c6e-a6c0-64b94f8f86ca
2,What advancements have been made by Anthropic ...,"Yo, Anthropic has been killing it in the LLM g...",None,"In 2024, Anthropic has made significant advanc...",0,0,1,2.953166,853aa0be-5e37-4597-8862-36a777f6dfd8,c9f44b91-6061-4623-b346-f0b21ddacfc7
3,What are the implications of Meta's advancemen...,None,"PermissionDeniedError(""Error code: 403 - {'err...","Meta's advancements in LLMs, particularly with...",0,1,1,0.065038,272e390e-00ef-49ea-986c-06fbfdbdbcc5,a04710a9-1982-4f07-8e96-0c74b314cb2f
4,What are the implications of gullibility in la...,"Yo, so check it out! The whole deal with gulli...",None,The implications of gullibility in language mo...,1,1,1,2.929178,942d4c8d-9671-4108-ae46-729a53f27d4f,26ff747b-737a-4c27-ab01-5ceb7279a487
5,What are the capabilities of LLMs in code gene...,"LLMs can write surprisingly competent code, an...",None,LLMs have demonstrated significant capabilitie...,1,1,1,2.486129,4541b378-2a89-4766-b0c1-4bd5417d6d58,20223a22-5528-404a-989e-3ef773c716cc
6,What are the code generation capabilities of L...,"Yo, here’s the scoop on LLMs and their code ga...",None,LLMs have demonstrated significant code genera...,1,0,1,4.956075,3f1458ed-31c0-4b18-b5dd-cfe5469e4d10,ba1d6c9f-6646-4eed-946e-ea242e9f263a
7,What are the code generation capabilities of L...,"Yo, check it! LLMs can whip up some code like ...",None,LLMs have demonstrated remarkable code generat...,1,0,1,4.415017,f9aa27f0-be79-4955-b783-06a603b07d52,1897025e-3356-4dcd-a7f0-d4e99832ca6b
8,What are the implications of DALL-E 3 on the f...,I don't know.,None,The implications of DALL-E 3 on the future of ...,0,0,0,0.698875,fe357357-2491-4c28-b9b2-8ab87dd13f3c,984bbdc7-cbcd-4ac8-a365-0d4d1e412f5e
9,What we know about GPT-4 and how it fit in LLMs?,"Yo, here's the lowdown on GPT-4 in the LLM sce...",None,"We don’t yet know how to build GPT-4, but it i...",0,0,1,4.362890,5115e8db-6966-49fd-a9d8-11a73ccfaed8,f33703a5-7e87-48c9-b3ae-a1f4cafe7b6f


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.


The screenshots show the results of two different RAG chains: the original and the "dope" version. Here's a breakdown of the differences and why certain metrics might have changed:

Correctness:
The "dope" chain seems to have improved correctness in some cases. This could be due to the more engaging prompt, which might help the model focus better on generating relevant answers.

Dopeness:
The "dope" chain naturally scores higher on dopeness because the prompt explicitly asks for a "cool" response. This aligns with the intended style of the output.

Helpfulness:
Helpfulness might not have improved as much because while the responses are more engaging, they may not always be more informative or accurate.

Latency:
The latency is slightly higher in the "dope" chain, possibly due to the increased complexity of generating more stylistically rich responses.

Errors:
There are some errors in the "dope" chain, which could be due to the model struggling with the new prompt style or the increased chunk size affecting retrieval accuracy.

Overall, the changes in metrics reflect the trade-offs between style and substance. The "dope" prompt enhances engagement but may not always improve factual accuracy or informativeness.
